# Process Economics Lesson: Optimizing a Distillation Column

**Objective:** This lesson demonstrates a core principle of engineering design: the trade-off between Capital Expenditure (CAPEX) and Operating Expenditure (OPEX). We will find the economically optimal design for a distillation column by varying the reflux ratio.

**Learning Goals:**
1.  Understand the relationship between reflux ratio, number of stages, and reboiler duty.
2.  Develop a simplified economic model for both CAPEX (column cost) and OPEX (energy cost).
3.  Combine an engineering model (McCabe-Thiele) with an economic model.
4.  Programmatically find the design point with the minimum Total Annualized Cost (TAC).
5.  Visualize and interpret the cost trade-off curve.

## The Economic Trade-Off

In distillation design, the **reflux ratio ($R$)** is the primary decision variable that drives the economics:

*   **Low Reflux Ratio (approaching $R_{min}$):**
    *   **OPEX:** Low. The energy required for the reboiler is small.
    *   **CAPEX:** High. An infinite number of stages are needed, making the column impossibly tall and expensive.

*   **High Reflux Ratio:**
    *   **OPEX:** High. A large amount of vapor must be boiled up to be sent back as reflux, consuming a lot of energy.
    *   **CAPEX:** Low. The number of stages required is minimal.

Somewhere between these two extremes lies an **optimal reflux ratio** that minimizes the **Total Annualized Cost (TAC)**.

## The Model: Combining Engineering and Economics

We will use the McCabe-Thiele algorithm as our engineering model to find the number of stages ($N$) for a given reflux ratio ($R$). Then, we'll apply a simple cost model:

**1. Capital Cost (CAPEX):** We'll assume the cost of the column is directly proportional to the number of theoretical stages.
$$ \text{CAPEX} = C_{stage} \cdot N $$

**2. Operating Cost (OPEX):** We'll assume the energy cost is proportional to the reboiler duty, which is itself proportional to the reflux ratio.
$$ \text{OPEX} = C_{energy} \cdot D \cdot (R+1) \cdot \Delta H_{vap} \cdot \text{Hours}_{op} $$

**3. Total Annualized Cost (TAC):**
$$ \text{TAC} = \frac{\text{CAPEX}}{\text{Payback Period}} + \text{OPEX} $$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# --- Engineering Parameters (same as separations lesson) ---
alpha = 2.5
zF = 0.50
xD = 0.95
xB = 0.05
q = 1.0

# --- Economic Parameters ---
D_flow = 100.0         # Distillate flow rate (kmol/hr)
H_vap = 35000         # Molar heat of vaporization (kJ/kmol)
cost_energy = 0.02    # Cost of energy ($/MJ)
cost_stage = 25000    # Installed cost per theoretical stage ($/stage)
hours_op = 8000       # Annual operating hours (hr/year)
payback_period = 3    # Payback period for capital (years)

print("Parameters defined.")

In [ ]:
def get_num_stages(R, alpha, zF, xD, xB, q):
    """A complete McCabe-Thiele function that returns the number of stages."""
    if abs(q - 1.0) < 1e-6:
        x_intersect = zF
    else:
        # For q != 1, solve for the intersection of q-line and ROL
        A_q = q / (q - 1)
        B_q = -zF / (q - 1)
        A_rol = R / (R + 1)
        B_rol = xD / (R + 1)
        x_intersect = (B_rol - B_q) / (A_q - A_rol)
    
    y_intersect = (R / (R + 1)) * x_intersect + (xD / (R + 1))
    
    # Check for invalid intersection which can happen if R < R_min
    if xB > x_intersect:
        return 200 # Return a large number for stages if R is too low
    
    m_sol = (y_intersect - xB) / (x_intersect - xB)
    
    num_stages = 0
    x_current, y_current = xD, xD
    while x_current > xB:
        num_stages += 1
        # Step horizontally to equilibrium curve
        x_next = y_current / (alpha - y_current * (alpha - 1))
        
        # Step vertically to the appropriate operating line
        if x_next > x_intersect:
            y_next = (R / (R + 1)) * x_next + (xD / (R + 1))
        else:
            y_next = m_sol * (x_next - xB) + xB
        
        x_current, y_current = x_next, y_next
        # Safety break to prevent infinite loops if R is too close to R_min
        if num_stages > 200:
            return 200
    return num_stages

# --- Find Minimum Reflux Ratio ---
y_eq_at_zF = alpha * zF / (1 + (alpha - 1) * zF)
R_min = (xD - y_eq_at_zF) / (y_eq_at_zF - zF)

# --- Run the Optimization Loop ---
reflux_ratios = np.linspace(R_min * 1.05, R_min * 4.0, 50) # Scan from 1.05*Rmin to 4*Rmin
results = []

for R_test in reflux_ratios:
    N = get_num_stages(R_test, alpha, zF, xD, xB, q)
    
    # Calculate Costs
    CAPEX = cost_stage * N
    # Reboiler duty Q = D*(R+1)*H_vap (in kJ/hr)
    reboiler_duty_MJ_hr = D_flow * (R_test + 1) * H_vap / 1000
    OPEX = reboiler_duty_MJ_hr * cost_energy * hours_op
    TAC = (CAPEX / payback_period) + OPEX
    
    results.append({'R': R_test, 'N': N, 'CAPEX': CAPEX, 'OPEX': OPEX, 'TAC': TAC})

print(f"Optimization complete. Minimum Reflux Ratio: {R_min:.2f}")

In [ ]:
# --- Plotting the Economic Results ---
R_plot = [res['R'] for res in results]
N_plot = [res['N'] for res in results]
CAPEX_plot = np.array([res['CAPEX'] for res in results]) / 1e6 # in M$
OPEX_plot = np.array([res['OPEX'] for res in results]) / 1e6 # in M$/year
TAC_plot = np.array([res['TAC'] for res in results]) / 1e6 # in M$/year

# Find the optimum point
min_tac_index = np.argmin(TAC_plot)
opt_R = R_plot[min_tac_index]
min_TAC = TAC_plot[min_tac_index]

plt.style.use('seaborn-v0_8-whitegrid')
fig, ax1 = plt.subplots(figsize=(12, 8))

ax1.plot(R_plot, CAPEX_plot / payback_period, 'b-o', label='Annualized CAPEX')
ax1.plot(R_plot, OPEX_plot, 'r-s', label='OPEX')
ax1.plot(R_plot, TAC_plot, 'k-', linewidth=3, label='Total Annualized Cost (TAC)')

# Highlight the optimum
ax1.axvline(x=opt_R, color='green', linestyle='--', label=f'Optimal R = {opt_R:.2f}')
ax1.plot(opt_R, min_TAC, 'g*', markersize=15)

# Formatting
ax1.set_xlabel('Reflux Ratio, R', fontsize=14)
ax1.set_ylabel('Annualized Cost (Million $/year)', fontsize=14)
ax1.set_title('Distillation Column Economic Optimization', fontsize=16, weight='bold')
ax1.legend()
ax1.set_ylim(bottom=0)
plt.show()

print(f"The optimal design occurs at a reflux ratio of {opt_R:.2f}, giving a minimum TAC of ${min_TAC:.3f} million/year.")

## Student Challenges

1.  **Cost of Energy:** The price of energy is volatile. What happens to the optimal reflux ratio if the `cost_energy` **doubles**? Re-run the simulation and explain the result. Does it make sense?
2.  **A Harder Separation:** An "easier" separation has a higher relative volatility. What happens to the overall costs if the separation is harder? Change `alpha` to `1.8` and see how all the costs are affected.